#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [24]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from tqdm import tqdm
from warnings import warn
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pickle

## Huangrui's Dataset

In [25]:
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
    
class BankDataset(StandardDataset):
    """financial-inclusion-in-africa dataset.
    """

    def __init__(self, path, label_name='y', favorable_classes=[1],  
                 protected_attribute_names=['age'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_drop=[],
                 features_to_keep=[],
                 na_values=[], custom_preprocessing=None,
                 metadata=None):
        """See :obj:`RegressionDataset` for a description of the arguments."""
        

        df = pd.read_csv(path)
        df["cons.conf.idx"] = -df["cons.conf.idx"]
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)

        super(BankDataset, self).__init__(
            df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

#### Load dataset and specify options

In [26]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [28]:
experiments_info = {}
budget = 0.01
for K in range(1, 6):
    dataset_orig_train= BankDataset(path="./Huangrui/bank/bank_train{}.csv".format(K))
    dataset_orig_test= BankDataset(path="./Huangrui/bank/bank_test{}.csv".format(K))
    #only use the budget% of the training data
    # Get the dataset and split into train and test
    if budget == 0.01:
        dataset_orig_train, _ = dataset_orig_train.split([0.01], shuffle=True,seed = 12345679)
    else:
        dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/bank'+str(K)+'_age_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
    sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
    dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    #自己计算error, 不是balanced accuracy！！！
    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

Balanced accuracy = 0.8191
Statistical parity difference = 0.3088
Disparate impact = 1.9426
Average odds difference = 0.1899
Equal opportunity difference = 0.0909
Theil index = 0.0626
Balanced accuracy = 0.8242
Statistical parity difference = 0.4216
Disparate impact = 2.0632
Average odds difference = 0.2222
Equal opportunity difference = 0.0000
Theil index = 0.0583
Balanced accuracy = 0.7788
Statistical parity difference = 0.2218
Disparate impact = 1.5046
Average odds difference = 0.1068
Equal opportunity difference = 0.0287
Theil index = 0.0675
Balanced accuracy = 0.7582
Statistical parity difference = 0.4338
Disparate impact = 1.8466
Average odds difference = 0.2530
Equal opportunity difference = 0.0259
Theil index = 0.0627
K = 1, budget = 0.01
The Error for the test dataset is 0.4035
The Equal opportunity difference for the test dataset is 0.02589


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\aif360\metrics\classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\aif360\metrics\classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,
C:\edb\languagepack\v2\Python-3.9\lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


IndexError: index 0 is out of bounds for axis 0 with size 0

# For budget = 0.01 We need to do it seperately

In [29]:
experiments_info = {}

In [38]:
dataset_orig_train= BankDataset(path="./Huangrui/bank/bank_train{}.csv".format(K))
dataset_orig_test= BankDataset(path="./Huangrui/bank/bank_test{}.csv".format(K))
K = 5
#only use the budget% of the training data
# Get the dataset and split into train and test
budget =0.01
   
dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=True)

# Lasso linear classifier and predictions
X_train = dataset_orig_train.features
y_train = dataset_orig_train.labels.ravel()
lmod = pickle.load(open('experiments/bank'+str(K)+'_age_bmodel.pkl','rb'))["clf"]

y_train_pred = lmod.predict(X_train)

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = dataset_orig_test_pred.features
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

#### Best threshold for classification only (no fairness)
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_train_pred.scores > class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
    
    classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                            dataset_orig_train_pred, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                    +classified_metric_orig_train.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

# print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
# print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
#### Estimate optimal parameters for the ROC method
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                privileged_groups=privileged_groups, 
                                low_class_thresh=0.01, high_class_thresh=0.99,
                                num_class_thresh=100, num_ROC_margin=50,
                                metric_name=metric_name,
                                metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
# print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
# print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

# Metrics for the train set
fav_inds = dataset_orig_train_pred.scores > best_class_thresh
dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

# display(Markdown("#### train set"))
# display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                unprivileged_groups, privileged_groups)

# Transform the validation set
dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

# display(Markdown("#### train set"))
# display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                unprivileged_groups, privileged_groups)
# Testing: Check if the metric optimized has not become worse
assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

# display(Markdown("#### Test set"))
# display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

# display(Markdown("#### Test set"))
# display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#自己计算error, 不是balanced accuracy！！！
print("K = {}, budget = {}".format(K, budget))
print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

Balanced accuracy = 0.9107
Statistical parity difference = -0.0639
Disparate impact = 0.7400
Average odds difference = -0.0411
Equal opportunity difference = 0.0000
Theil index = 0.0436
Balanced accuracy = 0.9107
Statistical parity difference = -0.0639
Disparate impact = 0.7400
Average odds difference = -0.0411
Equal opportunity difference = 0.0000
Theil index = 0.0436
Balanced accuracy = 0.8245
Statistical parity difference = 0.2034
Disparate impact = 1.6793
Average odds difference = 0.1225
Equal opportunity difference = 0.1062
Theil index = 0.0644
Balanced accuracy = 0.8245
Statistical parity difference = 0.2034
Disparate impact = 1.6793
Average odds difference = 0.1225
Equal opportunity difference = 0.1062
Theil index = 0.0644
K = 5, budget = 0.01
The Error for the test dataset is 0.2132
The Equal opportunity difference for the test dataset is 0.1062


In [39]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.3007543456871105,
  'Equal opportunity difference': 0.031039136302294157},
 'K = 2, budget = 0.01': {'Error': 0.4415286206330982,
  'Equal opportunity difference': 0.00544959128065392},
 'K = 3, budget = 0.01': {'Error': 0.42850114791734994,
  'Equal opportunity difference': 0.013495276653171406},
 'K = 4, budget = 0.01': {'Error': 0.25237782879632664,
  'Equal opportunity difference': 0.0625},
 'K = 5, budget = 0.01': {'Error': 0.21321961620469082,
  'Equal opportunity difference': 0.1061706629055007}}